### Vncorenlp

In [1]:
%%capture
!apt-get update && apt-get install openjdk-8-jdk
!pip -qq install  py-vncorenlp


In [2]:
import py_vncorenlp
import os
VNCORENLP_SAVEDIR=os.path.abspath("cache/vncorenlp")
os.makedirs(VNCORENLP_SAVEDIR,exist_ok=True)
py_vncorenlp.download_model(save_dir=VNCORENLP_SAVEDIR)

VnCoreNLP model folder /content/cache/vncorenlp already exists! Please load VnCoreNLP from this folder!


In [3]:
# Save the current working dir
previous_working_dir=os.path.abspath(os.getcwd())

# Khởi tạo VnCoreNLP với đầy đủ các annotators
# wseg: Tách từ
# pos: Gán nhãn từ loại
# ner: Nhận dạng thực thể
rdrsegmenter = py_vncorenlp.VnCoreNLP(
    annotators=["wseg", "pos", "ner"],
    save_dir=VNCORENLP_SAVEDIR
)

text = "Đại học Bách khoa Hà Nội là một trong những trường đại học hàng đầu tại Việt Nam."

# Chú ý: VnCoreNLP xử lý một list các câu.
# Kết quả trả về là một list các câu đã được xử lý.
# Mỗi câu là một list các từ, mỗi từ là một dictionary.
annotated_text = rdrsegmenter.annotate_text(text)
print(annotated_text)

# Lấy ra câu đầu tiên đã được xử lý (vì chúng ta chỉ có 1 câu đầu vào)
first_sentence = annotated_text[0]

print("--- TÁCH TỪ ---")
# form là từ đã được tách
# word_index là vị trí của từ
for i, word_info in enumerate(first_sentence):
    print(f"Từ {i}: {word_info['wordForm']}")


{0: [{'index': 1, 'wordForm': 'Đại_học', 'posTag': 'N', 'nerLabel': 'B-ORG', 'head': '_', 'depLabel': '_'}, {'index': 2, 'wordForm': 'Bách_khoa', 'posTag': 'Np', 'nerLabel': 'I-ORG', 'head': '_', 'depLabel': '_'}, {'index': 3, 'wordForm': 'Hà_Nội', 'posTag': 'Np', 'nerLabel': 'I-ORG', 'head': '_', 'depLabel': '_'}, {'index': 4, 'wordForm': 'là', 'posTag': 'V', 'nerLabel': 'O', 'head': '_', 'depLabel': '_'}, {'index': 5, 'wordForm': 'một', 'posTag': 'M', 'nerLabel': 'O', 'head': '_', 'depLabel': '_'}, {'index': 6, 'wordForm': 'trong', 'posTag': 'E', 'nerLabel': 'O', 'head': '_', 'depLabel': '_'}, {'index': 7, 'wordForm': 'những', 'posTag': 'L', 'nerLabel': 'O', 'head': '_', 'depLabel': '_'}, {'index': 8, 'wordForm': 'trường', 'posTag': 'N', 'nerLabel': 'O', 'head': '_', 'depLabel': '_'}, {'index': 9, 'wordForm': 'đại_học', 'posTag': 'N', 'nerLabel': 'O', 'head': '_', 'depLabel': '_'}, {'index': 10, 'wordForm': 'hàng_đầu', 'posTag': 'N', 'nerLabel': 'O', 'head': '_', 'depLabel': '_'}, {'

In [4]:
print("\n--- GÁN NHÃN TỪ LOẠI (POS) ---")
# posTag là nhãn từ loại
for word_info in first_sentence:
    print(f"{word_info['wordForm']:<15} {word_info['posTag']:<10}")


--- GÁN NHÃN TỪ LOẠI (POS) ---
Đại_học         N         
Bách_khoa       Np        
Hà_Nội          Np        
là              V         
một             M         
trong           E         
những           L         
trường          N         
đại_học         N         
hàng_đầu        N         
tại             E         
Việt_Nam        Np        
.               CH        


In [5]:
# VnCoreNLP sẽ gán nhãn NER cho từng từ. Chúng ta cần nhóm các từ lại với nhau.
print("\n--- NHẬN DẠNG THỰC THỂ (NER) ---")

# Các biến để lưu trữ thực thể đang được xử lý
current_entity_words = []
current_entity_label = "O"  # "O" nghĩa là Outside (không phải thực thể)

for word_info in first_sentence:
    word = word_info['wordForm']
    label = word_info['nerLabel']

    if label.startswith("B-"):  # B- là Begin (bắt đầu một thực thể mới)
        # Nếu có thực thể cũ, in nó ra trước khi bắt đầu thực thể mới
        if current_entity_words:
            entity_text = ' '.join(current_entity_words).replace('_', ' ')
            print(f"{entity_text:<30} {current_entity_label:<15}")

        # Bắt đầu thực thể mới
        current_entity_words = [word]
        current_entity_label = label[2:]  # Bỏ "B-" đi để lấy nhãn (ví dụ: B-ORG -> ORG)

    elif label.startswith("I-") and label[2:] == current_entity_label:  # I- là Inside (bên trong thực thể)
        # Nếu từ này là một phần của thực thể hiện tại, thêm nó vào
        current_entity_words.append(word)

    else:  # Nếu là nhãn "O" hoặc một thực thể mới bắt đầu không liên quan
        # Nếu có thực thể cũ
        if current_entity_words:
            entity_text = ' '.join(current_entity_words).replace('_', ' ')
            print(f"{entity_text:<30} {current_entity_label:<15}")

        # Reset
        current_entity_words = []
        current_entity_label = "O"

if current_entity_words:
    entity_text = ' '.join(current_entity_words).replace('_', ' ')
    print(f"{entity_text:<30} {current_entity_label:<15}")


--- NHẬN DẠNG THỰC THỂ (NER) ---
Đại học Bách khoa Hà Nội       ORG            
Việt Nam                       LOC            
